In [1]:
import cv2
import os
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras import optimizers
from keras.applications import MobileNet
from keras.backend.tensorflow_backend import set_session
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

keras.backend.clear_session()
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Using TensorFlow backend.
/home/zsvizi/workspace/EFOP/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zsvizi/workspace/EFOP/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zsvizi/workspace/EFOP/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zsvizi/workspace/EFO

In [ ]:
def preprocess_images():
    path_v_r_r = './data/validation_original/realfoggy/'
    path_v_s_r = './data/validation_original/syntheticfoggy/'
    path_t_r_r = './data/train_original/realfoggy/'
    path_t_s_r = './data/train_original/syntheticfoggy/'
    paths_read = [path_v_r_r, path_v_s_r, path_t_r_r, path_t_s_r]
    path_v_r_w = './data/validation/realfoggy/'
    path_v_s_w = './data/validation/syntheticfoggy/'
    path_t_r_w = './data/train/realfoggy/'
    path_t_s_w = './data/train/syntheticfoggy/'
    paths_write = [path_v_r_w, path_v_s_w, path_t_r_w, path_t_s_w]
    labels = ['realfoggy', 'syntheticfoggy', 'realfoggy', 'syntheticfoggy']

    for (path_read, path_write, label) in zip(paths_read, paths_write, labels):
        counter = 0
        for filename in os.listdir(path_read):
            img = cv2.imread(os.path.join(path_read,filename))
            img = cv2.resize(img,(398, 224))
            img = img[:, 87:311, :]
            cv2.imwrite(path_write + label + str(counter) + '.jpg', img)
            counter += 1
    # if we want to show any of the images
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
# https://colab.research.google.com/drive/1s1lkOiqKgl9_MsXJtPF--oh6uXHYkNYm#scrollTo=6IgZ7G0r4mUt
def change_model(model, new_input_shape=(None, 40, 40, 3)):
    # replace input shape of first layer
    model._layers[0].batch_input_shape = new_input_shape

    # rebuild model architecture by exporting and importing via json
    new_model = keras.models.model_from_json(model.to_json())

    # copy weights from old model to new one
    for layer in new_model.layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
            print("Loaded layer {}".format(layer.name))
        except:
            print("Could not transfer weights for layer {}".format(layer.name))

    return new_model

In [2]:
# load model without classifier layers
extractor = MobileNet(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
# add new classifier layers
model = Sequential()
model.add(extractor)
# model.add(Flatten(input_shape=model.output_shape[1:]))
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
for layer in extractor.layers[:25]:
    layer.trainable = False
# define new model
# summarize
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1025      
Total params: 4,279,489
Trainable params: 4,226,945
Non-trainable params: 52,544
_________________________________________________________________


In [3]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2 * 3088
nb_validation_samples = 2 * 720
epochs = 50
batch_size = 16

if keras.backend.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [6]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

# fine-tune the model
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Found 6176 images belonging to 2 classes.
Found 1440 images belonging to 2 classes.
Epoch 1/50
386/386 [==============================] - 65s 170ms/step - loss: 0.3199 - accuracy: 0.8632 - val_loss: 0.0799 - val_accuracy: 0.9875
Epoch 2/50
386/386 [==============================] - 60s 155ms/step - loss: 0.0950 - accuracy: 0.9722 - val_loss: 0.0240 - val_accuracy: 0.9958
Epoch 3/50
386/386 [==============================] - 60s 155ms/step - loss: 0.0595 - accuracy: 0.9824 - val_loss: 0.0169 - val_accuracy: 0.9986
Epoch 4/50
 53/386 [===>..........................] - ETA: 42s - loss: 0.0580 - accuracy: 0.9776

KeyboardInterrupt: 

In [ ]:
model.save_weights('weights.h5')